### LagLlama
https://github.com/time-series-foundation-models/lag-llama/tree/main


https://colab.research.google.com/drive/1DRAzLUPxsd-0r8b-o4nlyFXrjw_ZajJJ?usp=sharing

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd '/content/drive/MyDrive/THESIS/rag-thesis/'

/content/drive/MyDrive/THESIS/rag-thesis


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from forecasting_utils import *


In [7]:
!git clone -b update-gluonts https://github.com/time-series-foundation-models/lag-llama/

Cloning into 'lag-llama'...
remote: Enumerating objects: 489, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 489 (delta 149), reused 109 (delta 109), pack-reused 312 (from 3)
Receiving objects: 100% (489/489), 277.77 KiB | 1.93 MiB/s, done.
Resolving deltas: 100% (247/247), done.


In [4]:
cd '/content/drive/MyDrive/THESIS/rag-thesis/lag-llama'

/content/drive/MyDrive/THESIS/rag-thesis/lag-llama


In [5]:
!pip install -q -U -r requirements.txt  # this could take some time # ignore the errors displayed by colab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**Restart your runtime now, and then continue**

In [6]:
cd '/content/drive/MyDrive/THESIS/rag-thesis/lag-llama'

/content/drive/MyDrive/THESIS/rag-thesis/lag-llama


In [7]:
!huggingface-cli download time-series-foundation-models/Lag-Llama lag-llama.ckpt --local-dir /content/drive/MyDrive/THESIS/rag-thesis/lag-llama

lag-llama.ckpt: 100% 29.5M/29.5M [00:00<00:00, 53.1MB/s]
Download complete. Moving file to /content/drive/MyDrive/THESIS/rag-thesis/lag-llama/lag-llama.ckpt
/content/drive/MyDrive/THESIS/rag-thesis/lag-llama/lag-llama.ckpt


In [7]:
from itertools import islice

from matplotlib import pyplot as plt
import matplotlib.dates as mdates

import torch
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.repository.datasets import get_dataset

from gluonts.dataset.pandas import PandasDataset
import pandas as pd

from lag_llama.gluon.estimator import LagLlamaEstimator

In [8]:
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error, r2_score
def run_ar_benchmark(train_data, test_data, target_var, ar_order=1, horizon=1):
    """
    Runs an AR(p) model on the transformed dataset and returns performance metrics.

    Args:
        train_data (pd.DataFrame): Training data
        test_data (pd.DataFrame): Test data
        target_var (str): Target variable name
        ar_order (int): Order of the AR model
        horizon (int): Number of future points to forecast
    """
    # Ensure the index has frequency information
    if not isinstance(train_data.index, pd.DatetimeIndex) or train_data.index.freq is None:
        train_data.index = pd.date_range(
            start=train_data.index[0],
            periods=len(train_data),
            freq='Q'
        )

    # Fit AR(p) model
    model = AutoReg(train_data[target_var], lags=ar_order)
    model_fit = model.fit()

    # Make predictions for the next h points
    predictions = model_fit.forecast(steps=horizon)

    # Get the actual values for the first h points of test data
    actuals = test_data[target_var][:horizon]

    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    return rmse


In [9]:
import sys
from types import ModuleType

# Create dummy module hierarchy
def create_dummy_module(module_path):
    """
    Create a dummy module hierarchy for the given path.
    Returns the leaf module.
    """
    parts = module_path.split('.')
    current = ''
    parent = None

    for part in parts:
        current = current + '.' + part if current else part
        if current not in sys.modules:
            module = ModuleType(current)
            sys.modules[current] = module
            if parent:
                setattr(sys.modules[parent], part, module)
        parent = current

    return sys.modules[module_path]

# Create the dummy gluonts module hierarchy
gluonts_module = create_dummy_module('gluonts.torch.modules.loss')

# Create dummy classes for the specific loss functions
class DistributionLoss:
    def __init__(self, *args, **kwargs):
        pass

    def __call__(self, *args, **kwargs):
        return 0.0

    def __getattr__(self, name):
        return lambda *args, **kwargs: None

class NegativeLogLikelihood:
    def __init__(self, *args, **kwargs):
        pass

    def __call__(self, *args, **kwargs):
        return 0.0

    def __getattr__(self, name):
        return lambda *args, **kwargs: None

# Add the specific classes to the module
gluonts_module.DistributionLoss = DistributionLoss
gluonts_module.NegativeLogLikelihood = NegativeLogLikelihood

In [10]:
def get_lag_llama_predictions(dataset, prediction_length, device, context_length=32, use_rope_scaling=False, num_samples=100):
    ckpt = torch.load("lag-llama.ckpt", map_location=device, weights_only=False) # Uses GPU since in this Colab we use a GPU.
    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

    rope_scaling_arguments = {
        "type": "linear",
        "factor": max(1.0, (context_length + prediction_length) / estimator_args["context_length"]),
    }

    estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length, # Lag-Llama was trained with a context length of 32, but can work with any context length

        # estimator args
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        scaling=estimator_args["scaling"],
        time_feat=estimator_args["time_feat"],
        rope_scaling=rope_scaling_arguments if use_rope_scaling else None,

        batch_size=1,
        num_parallel_samples=100,
        device=device,
    )

    lightning_module = estimator.create_lightning_module()
    transformation = estimator.create_transformation()
    predictor = estimator.create_predictor(transformation, lightning_module)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset,
        predictor=predictor,
        num_samples=num_samples
    )
    forecasts = list(forecast_it)
    tss = list(ts_it)

    return forecasts, tss

In [11]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [14]:
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
def run_lagllama(df, ratio=0.7):
    variables = list(df.select_dtypes(include=[np.number]).columns)
    num_samples = 100 # number of samples sampled from the probability distribution for each timestep
    device = torch.device("cuda:0") # You can switch this to CPU or other GPUs if you'd like, depending on your environment

    # Convert numerical columns to float 32 format for lag-llama
    df_wide = df.copy()
    for col in df_wide.columns:
        # Check if column is not of string type
        if df_wide[col].dtype != 'object' and pd.api.types.is_string_dtype(df_wide[col]) == False:
            df_wide[col] = df_wide[col].astype('float32')

    # Horizons to evaluate
    horizons = [1,3,6,12]
    results = {
        h: {
            'RMSE': [],
            'cum_RMSE': {var:[] for var in variables}
        } for h in ["h=1", "h=3", "h=6", "h=12"]
    }

    for h in horizons:
        print(f"Running for horizon {h}...")

        prediction_length = h  # your forecast horizon

        test_idx = 67
        df_truncated = df_wide.iloc[:-test_idx]
        ds = PandasDataset(dict(df_truncated), freq='Q')

        # Test train split for AR data (wide format)
        split_idx = 180
        split_date = df.index[split_idx]  # This is the split point

        df_ar = df.copy()
        train_data_ar = df_ar.iloc[:split_idx]
        test_data_ar = df_ar.iloc[split_idx:]

        # Calculate RMSE and statistics for each variable
        current_cum_sums ={var:0 for var in variables}
        rmses = {var:[] for var in variables}  # Store all RMSE values for this horizon
        ar_rmses = {var:[] for var in variables}

        while test_idx > 13:
            forecasts, tss = get_lag_llama_predictions(ds, prediction_length, device, context_length=32, num_samples=500)

            # Add rolling window RMSEs to rmses dict for each variable
            for forecast, actual_series in zip(forecasts, tss):

                y_pred = forecast.mean                # (prediction_length,)
                # Slice the actual series to match the prediction length
                y_true = actual_series.values[-prediction_length:]  # Get the last 'prediction_length' values

                rmse = np.sqrt(mean_squared_error(y_true, y_pred))
                rmses[forecast.item_id].append(rmse)

                current_cum_sums[forecast.item_id] += rmse
                results["h="+str(h)]['cum_RMSE'][forecast.item_id].append(current_cum_sums[forecast.item_id])

            test_idx -= 1
            df_truncated = df_wide.iloc[:-67]



        while split_idx < 245:
            # Calculate metrics for each unique_id (variable)
            for var in variables:
                ar1_rmse = run_ar_benchmark(train_data_ar, test_data_ar, var, ar_order=1, horizon=h)
                ar_rmses[var].append(ar1_rmse)

            # Shift test-train split by 1 timestamp
            split_idx += 1
            train_data_ar = df_ar.iloc[:split_idx]
            test_data_ar = df_ar.iloc[split_idx:]



        for var in variables:
            # Compute root mean square error for each variable
            results["h="+str(h)]['RMSE'].append(np.sum(rmses[var])/np.sum(ar_rmses[var]))

    return results


df = pd.read_csv('/content/drive/MyDrive/THESIS/rag-thesis/data/processed_FRED_data.csv', index_col=0, parse_dates=True)
df.index = pd.to_datetime(df.index)
lagllama_results = run_lagllama(df)

# Print results
print("\nDetailed RMSE for each variable and horizon:")
print(lagllama_results)


Running for horizon 1...
Running for horizon 3...
Running for horizon 6...
Running for horizon 12...

Detailed RMSE for each variable and horizon:
{'h=1': {'RMSE': [np.float64(4.4831754888124), np.float64(1.0833784562309343), np.float64(0.9073918297063973), np.float64(2.0969239047498887), np.float64(0.5346949801562733), np.float64(0.1427553723513558), np.float64(2.205021685328502), np.float64(2.541059871859689), np.float64(0.03127824604368388), np.float64(1.8011869749035143), np.float64(3.50066917138237), np.float64(0.8835861159843279), np.float64(1.4559690248492263), np.float64(0.7916293661499296), np.float64(0.12460579257485949), np.float64(0.8273015749916002), np.float64(0.1880983354270705), np.float64(1.3301546818746695), np.float64(0.7275974337893724), np.float64(1.015412267645198)], 'cum_RMSE': {'GDP_log': [np.float64(0.04446268177548964), np.float64(0.08247423339661389), np.float64(0.11993837504810577), np.float64(0.16425871975179124), np.float64(0.20458012975855588), np.float64

In [25]:
for h in lagllama_results:
    print(f"Horizon {h}:")
    rmse_array = np.array(lagllama_results[h]['RMSE'])
    rmse_array = list(rmse_array[rmse_array <= 3])
    lagllama_results[h]['RMSE'] = rmse_array
    # Calculate statistics for this horizon
    print("Mean:", np.mean(rmse_array))
    print("Std: ", np.std(rmse_array))
    print("Min: ", np.min(rmse_array))
    print("Max: ", np.max(rmse_array))
    print("Q25: ", np.percentile(rmse_array, 25))
    print("Median: ", np.percentile(rmse_array, 50))
    print("Q75: ", np.percentile(rmse_array, 75))
    print("IQR: ", np.percentile(rmse_array, 75) - np.percentile(rmse_array, 25))

Horizon h=1:
Mean: 1.0382247730342498
Std:  0.7247511969912033
Min:  0.03127824604368388
Max:  2.541059871859689
Q25:  0.5829205935645481
Median:  0.8954889728453626
Q75:  1.424515439105587
IQR:  0.8415948455410389
Horizon h=3:
Mean: 1.077146344831501
Std:  0.8400591148670653
Min:  0.1360653292469759
Max:  2.6416081318926135
Q25:  0.34626820093635535
Median:  0.6993956095959599
Q75:  1.8187600273822013
IQR:  1.472491826445846
Horizon h=6:
Mean: 0.990384350712221
Std:  0.784040198709196
Min:  0.17539606290753665
Max:  2.4043298814624356
Q25:  0.4072066399566362
Median:  0.6636946930869321
Q75:  1.6305505216339733
IQR:  1.2233438816773372
Horizon h=12:
Mean: 0.9934598305998766
Std:  0.8540753937014735
Min:  0.15029659594301967
Max:  2.72469678805618
Q25:  0.41827787039485487
Median:  0.6369022464478201
Q75:  1.0508271526030926
IQR:  0.6325492822082377


In [26]:
import json
with open('../results/lagllama_results.json', 'w') as f:
    json.dump(lagllama_results, f,indent=4,)